In [10]:
import pandas as pd
import numpy as np
from pathlib import Path
from IPython.display import display, HTML
PATH_CACHE = Path("../../data/algos_results.csv").resolve()
print(PATH_CACHE)

C:\Users\robin\Documents\Cours\Poly\Recherche\MaitriseClustering\data\algos_results.csv


In [53]:
df = pd.read_csv(PATH_CACHE)

In [54]:
df100 = df[df["init_type"] == "kmean+"]
print(df100["num_clusters"].unique())
Lcluster_concat = []
for num_clusters in [4,8,16,64]:
    Linit_concat = []
    for ImprClass, iteration_order in list(zip(["CallbackBestImprovement", "CallbackFirstImprovement","CallbackFirstImprovement","CallbackFirstImprovement"], ["BACK","CURR", "BACK", "RANDOM"])):
        tmp_df = df100[df100["ImprClass"] == ImprClass]
        tmp_df = tmp_df[tmp_df["iteration_order"] == iteration_order].sort_values(by=["uuid_points_coords","uuid_points_assign"],axis=0)
        tmp_df = tmp_df[tmp_df["num_clusters"] == num_clusters]
        tmp_df = tmp_df.drop(["init_cost","ImprClass","init_type","iteration_order","num_clusters","uuid_points_coords","uuid_points_assign"],axis=1)
        tmp_df.reset_index(drop=True, inplace=True)
        if ImprClass == "CallbackBestImprovement":
            backup_tmp_df = tmp_df.copy()
        else:
            tmp_df["cost_augm"] = tmp_df["end_cost"].reset_index(drop=True) - backup_tmp_df["end_cost"].reset_index(drop=True)
            # tmp_df = tmp_df.rename(columns={"end_cost":"cost_augm_BI"})
        tmp_df = tmp_df.groupby("num_points",as_index=False).mean()
        tmp_df = tmp_df.set_index(['num_points'])
        Linit_concat.append(tmp_df)
    
    Lcluster_concat.append(pd.concat(Linit_concat,axis=1, keys=["BI_BACK", "FI_CURR", "FI_BACK", "FI_RANDOM"]))

[16  2  4  8 32 64]


In [55]:

# for df,num_clusters in zip(Lcluster_concat, [4,8,16,64]):
#     pd.set_option('display.max_rows', None, 'display.max_columns', None)
#     print(f"{num_clusters} clusters")
#     print(display(df.head()))

In [56]:
pd.set_option('display.max_rows', None, 'display.max_columns', None)
df100_cluster = pd.concat(Lcluster_concat, axis=1, keys=["4 clusters", "8 clusters","16 clusters", "64 clusters"])
df100_cluster

4 clusters                                                      \
                 BI_BACK                          FI_CURR                      
                end_cost num_iter      time      end_cost num_iter      time   
num_points                                                                     
20           5666.104084    4.566  0.047500   5708.917922    6.231  0.039417   
30           9191.637699    9.742  0.148194   9271.308567   11.730  0.110792   
40          14652.902367   14.100  0.268915  14647.878613   18.210  0.215448   
50          18844.731802   16.393  0.362979  18695.749330   21.598  0.276548   
60          22199.490576   21.021  0.557036  22162.996923   27.006  0.414796   
70          28808.300117   23.858  0.729477  28414.490180   32.273  0.560921   
80                   NaN      NaN       NaN           NaN      NaN       NaN   

                                                                     \
                             FI_BACK                                  
             cost_augm      end_cost num_iter      time   cost_augm   
num_points                                                            
20           42.813838   5713.990093    6.204  0.033069   47.886009   
30           79.670867   9271.308565   11.696  0.094534   79.670866   
40           -5.023754  14656.341074   18.238  0.183205    3.438707   
50         -148.982471  18702.612748   21.615  0.234178 -142.119054   
60          -36.493653  22161.020119   27.023  0.351163  -38.470457   
70         -393.809937  28414.509838   32.203  0.475477 -393.790279   
80                 NaN           NaN      NaN       NaN         NaN   

                                                           8 clusters  \
               FI_RANDOM                                      BI_BACK   
                end_cost num_iter      time   cost_augm      end_cost   
num_points                                                              
20           5716.541750    6.265  0.034843   50.437666   1845.740767   
30           9271.308566   11.691  0.099346   79.670867   3989.666723   
40          14665.506216   18.168  0.189612   12.603849   6233.574676   
50          18687.750081   21.563  0.244566 -156.981721   8700.293555   
60          22167.579978   27.049  0.368162  -31.910598  10927.631942   
70          28403.214887   32.182  0.497738 -405.085230  12657.740678   
80                   NaN      NaN       NaN         NaN           NaN   

                                                                            \
                                    FI_CURR                                  
           num_iter      time      end_cost num_iter      time   cost_augm   
num_points                                                                   
20            5.723  0.131965   1905.375321    8.594  0.105436   59.634554   
30            9.422  0.320688   4000.296804   15.921  0.258423   10.630082   
40           15.058  0.656456   6178.820095   26.751  0.563169  -54.754581   
50           19.414  0.961869   8561.910829   35.633  0.836682 -138.382726   
60           26.713  1.561256  10431.772171   52.690  1.534050 -495.859772   
70           34.533  2.525936  12524.753321   63.392  2.248608 -132.987357   
80              NaN       NaN           NaN      NaN       NaN         NaN   

                                                                       \
                 FI_BACK                                    FI_RANDOM   
                end_cost num_iter      time   cost_augm      end_cost   
num_points                                                              
20           1906.406861    8.698  0.092192   60.666094   1908.470152   
30           3997.765533   16.036  0.224805    8.098810   4001.270119   
40           6191.894092   26.548  0.481445  -41.680584   6190.743093   
50           8557.949586   35.809  0.725162 -142.343969   8553.919495   
60          10455.116078   52.700  1.320093 -472.515864  10436.501849   
70          12497.626232   63.4